
## Installing the CUDA Toolkit and `nvcc`

<img src="nvcc.svg" alt="nvcc" width="500" height="300">


To compile and run CUDA code, you need the NVIDIA CUDA Toolkit installed on your system. The toolkit includes the NVIDIA CUDA Compiler (`nvcc`), essential libraries, and header files (like `<cuda/std/mdspan>`).

**1. The Recommended Method: Install the CUDA Toolkit**

The most straightforward way to get `nvcc` and all necessary components is to install the official CUDA Toolkit from NVIDIA.

*   **Go to the NVIDIA CUDA Toolkit Download Page:** [https://developer.nvidia.com/cuda-downloads](https://developer.nvidia.com/cuda-downloads)
*   **Select your System:** Choose your Linux distribution (e.g., Ubuntu, Fedora, RHEL), architecture (e.g., x86_64), and preferred installer type (package manager recommended for ease of updates).
*   **Follow the Provided Instructions:** NVIDIA provides detailed instructions for each system and installer type. Using your distribution's package manager (like `apt` for Debian/Ubuntu or `dnf`/`yum` for Fedora/RHEL/Rocky) is often the easiest method.

**Example (Ubuntu using package manager):** The instructions might involve adding an NVIDIA repository and then running commands similar to:

```bash
sudo apt-get update
# Replace <version> e.g., cuda-toolkit-12-5
sudo apt-get install cuda-toolkit-<version>
```

A standard installation (especially via package managers or to the default `/usr/local/cuda` location) typically sets up the necessary environment variables (`PATH` and `LD_LIBRARY_PATH`) automatically, so `nvcc` and its associated headers/libraries should be found.

**2. Verifying the Installation**

After installation, open a *new* terminal window (to ensure environment variables are loaded) and check the `nvcc` version:

```bash
nvcc --version
```

This should output the compiler version information if the installation was successful and `nvcc` is in your system's `PATH`.

**3. Troubleshooting: Missing Headers (`cuda/std/mdspan`) or `nvcc` Not Found**

Sometimes, especially if you have multiple CUDA versions or installed to a custom location (e.g., using Spack), the environment might not pick up the correct installation automatically.

*   **Problem:** Compiling code (often within a Jupyter Notebook or specific environment) fails with errors like `fatal error: cuda/std/mdspan: No such file or directory` or commands like `!nvcc ...` fail because `nvcc` isn't found.
*   **Reason:** This usually means either the compiler isn't finding the header files from the correct CUDA Toolkit installation, or the `nvcc` executable itself isn't in the environment's `PATH`. The `<cuda/std/mdspan>` header is part of the `libcudacxx` library, which is included in modern CUDA Toolkits. ([libcudacxx documentation](https://nvidia.github.io/cccl/libcudacxx/setup/getting.html))

*   **Solution 1: Ensure Correct `nvcc` is in `PATH` (Especially for Notebooks)**
    If you know where your desired CUDA Toolkit is installed, you can explicitly add its `bin` directory to the `PATH` environment variable *within your session or script/notebook*. This ensures the correct `nvcc` (and by extension, its associated environment) is used.

    **Example (Python/Jupyter Notebook):**
    
    ```python
    import os
    import sys

    # --- ADJUST THIS PATH ---
    # Example path for a CUDA installation via Spack (replace with YOUR actual path)
    # cuda_bin_path = "/packages/apps/spack/21/opt/spack/linux-rocky8-zen3/gcc-12.1.0/cuda-12.6.1-cf4xlcbcfpwchqwo5bktxyhjagryzcx6/bin"

    # Example path for a default installation (check if '/usr/local/cuda/bin' exists)
    cuda_bin_path = "/usr/local/cuda/bin" # MODIFY THIS IF NEEDED

    # Check if the path exists before adding
    if os.path.isdir(cuda_bin_path):
        # Prepend the CUDA bin path to the environment PATH if not already present
        if cuda_bin_path not in os.environ['PATH']:
            print(f"Adding {cuda_bin_path} to PATH")
            os.environ['PATH'] = cuda_bin_path + os.pathsep + os.environ['PATH']
        else:
            print(f"{cuda_bin_path} already in PATH")
    else:
        print(f"Warning: Specified CUDA bin path does not exist: {cuda_bin_path}")

    # Optional: Verify nvcc is found now (within the notebook's context)
    # The exact command might depend on your notebook setup (! or %sx)
    print("\nVerifying nvcc location:")
    !which nvcc
    print("\nVerifying nvcc version:")
    !nvcc --version
    ```
    
    **Important:** Replace the `cuda_bin_path` value with the *actual path* to the `bin` directory inside your CUDA Toolkit installation. Verify the path exists.

*   **Solution 2: Explicit Include Paths (Command-Line Compilation)**
    If compiling directly with `nvcc` on the command line and headers aren't found, you might need to explicitly tell the compiler where to look using the `-I` flag. The `libcudacxx` headers are usually in a subdirectory within the toolkit.

    ```bash
    # Example: Adjust the path '/usr/local/cuda-12.5' to your actual installation
    # The exact path for libcudacxx headers might vary slightly between versions
    nvcc your_code.cu -o your_executable -I/usr/local/cuda-12.5/include/
    # Or potentially deeper for libcudacxx specifically if needed:
    # nvcc your_code.cu -o your_executable -I/usr/local/cuda-12.5/include/libcudacxx/include/
    ```
    
    *(Note: This is less common for the standard library headers like `<cuda/std/mdspan>` if `nvcc` itself is found correctly via `PATH`, but can be useful for other libraries or troubleshooting).*

**4. Alternative: Using NVIDIA NGC Containers**

For a potentially simpler setup, consider using NVIDIA NGC containers ([https://catalog.ngc.nvidia.com/](https://catalog.ngc.nvidia.com/)). These are pre-built Docker containers with the CUDA Toolkit, drivers, and common AI/HPC frameworks already installed and configured, avoiding local installation complexities.

**5. Note on Linux Kernel Headers**

You might see recommendations to install `linux-headers-$(uname -r)`. These are necessary for building kernel modules, specifically for installing the NVIDIA *driver*. While the driver is required to *run* CUDA applications, installing kernel headers doesn't directly provide the CUDA Toolkit (`nvcc`) or its C++ library headers like `mdspan`. You typically need both the driver *and* the toolkit installed. Installing the toolkit usually handles its own dependencies.
